<a href="https://colab.research.google.com/github/feuerteufelfw/SOC/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:

import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from itertools import product
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
gpus = tf.config.list_physical_devices('GPU')
print("Verwendete Geräte:", tf.config.list_physical_devices('GPU'))


# === Daten laden ===
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
y_train, y_test = y_train.flatten(), y_test.flatten()
mask_train, mask_test = np.isin(y_train, [0,1]), np.isin(y_test, [0,1])
x_train, y_train = x_train[mask_train]/255., y_train[mask_train]
x_test, y_test = x_test[mask_test]/255., y_test[mask_test]
y_train_o, y_test_o = to_categorical(y_train,2), to_categorical(y_test,2)

# === Parameter ===
dropouts = [ 0.3]
learning_rates = [1e-4]
batch_sizes = [8]
filters_list = [(32,64), (64,128)]  # mehrschichtig
kernel_sizes = [(3,3), (5,5)]  # verschiedene Kernelgrößen
optimizers = ['adam', 'sgd', 'rmsprop', 'adagrad']
epochs = 30

# === Model-Builder ===
def build_model(dp, lr, filters, kernel_size, optimizer_name):
    optimizer_dict = {
        'adam': tf.keras.optimizers.Adam(lr),
        'sgd': tf.keras.optimizers.SGD(lr),
        'rmsprop': tf.keras.optimizers.RMSprop(lr),
        'adagrad': tf.keras.optimizers.Adagrad(lr)
    }
    model = Sequential([
        Conv2D(filters[0], kernel_size, activation='relu',padding='same', input_shape=(32,32,3)),
        BatchNormalization(),
        MaxPooling2D(),
        Conv2D(filters[1], kernel_size, activation='relu',padding='same'),
        BatchNormalization(),
        MaxPooling2D(),
        Flatten(),
        Dropout(dp),
        Dense(64, activation='relu'),  # größere Dense-Schicht
        Dropout(dp),
        Dense(2, activation='softmax')
    ])
    model.compile(optimizer=optimizer_dict[optimizer_name], loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# === EarlyStopping ===
es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# === Brute-Force intelligent (kleine Parameterwahl) ===
results = []
for dp, lr, bs, filters, ks, opt in product(dropouts, learning_rates, batch_sizes, filters_list, kernel_sizes, optimizers):
    print(f'Testing: dp={dp}, lr={lr}, bs={bs}, filters={filters}, kernel_size={ks}, optimizer={opt}')
    model = build_model(dp, lr, filters, ks, opt)
    history = model.fit(x_train, y_train_o,
                        epochs=epochs,
                        batch_size=bs,
                        validation_data=(x_test, y_test_o),
                        callbacks=[es], verbose=1)
    acc = model.evaluate(x_test, y_test_o, verbose=1)[1]
    epochs_run = len(history.history['loss'])
    results.append((dp, lr, bs, filters, ks, opt, acc, epochs_run))
    print(f'--> accuracy: {acc:.4f} after {epochs_run} epochs')

# === Bestes Ergebnis ===
best = max(results, key=lambda x: x[-2])
print('\nBeste Kombination:', best)

Num GPUs Available: 1
Verwendete Geräte: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Testing: dp=0.3, lr=0.0001, bs=8, filters=(32, 64), kernel_size=(3, 3), optimizer=adam
Epoch 1/30
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.7641 - loss: 0.5921 - val_accuracy: 0.9050 - val_loss: 0.2433
Epoch 2/30
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8879 - loss: 0.2697 - val_accuracy: 0.9220 - val_loss: 0.1951
Epoch 3/30
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9089 - loss: 0.2228 - val_accuracy: 0.9275 - val_loss: 0.1646
Epoch 4/30
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9289 - loss: 0.1738 - val_accuracy: 0.9320 - val_loss: 0.1640
Epoch 5/30
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9352 - loss: 0.1536 - val_accuracy: 0.9360 - val_loss: 0.1488
Epoch 6/30
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9482 - loss: 0.1347 - val_accuracy: 0.9375 - val_loss: 0.1394
Epoch 7/30
1250/1250 ━━━━

In [ ]:
dropout=0.3,
learning_rate=0.0001
batch_size=8
filters=(32, 64)
kernel_size=(3, 3)
optimizer='adam'
accuracy=0.9575
epochs_run=18
print(f'Testing: dp={dp}, lr={lr}, bs={bs}, filters={filters}, kernel_size={ks}, optimizer={opt}')
model = build_model(dp, lr, filters, ks, opt)
history = model.fit(x_train, y_train_o,
                    epochs=epochs,
                    batch_size=bs,
                    validation_data=(x_test, y_test_o),
                    callbacks=[es], verbose=1)
acc = model.evaluate(x_test, y_test_o, verbose=1)[1]
epochs_run = len(history.history['loss'])
results.append((dp, lr, bs, filters, ks, opt, acc, epochs_run))
print(f'--> accuracy: {acc:.4f} after {epochs_run} epochs')

Testing: dp=0.3, lr=0.0001, bs=8, filters=(32, 64), kernel_size=(5, 5), optimizer=adagrad
Epoch 1/30
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.6675 - loss: 0.7726 - val_accuracy: 0.8250 - val_loss: 0.3985
Epoch 2/30
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.7563 - loss: 0.5502 - val_accuracy: 0.8380 - val_loss: 0.3713
Epoch 3/30
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7749 - loss: 0.4902 - val_accuracy: 0.8430 - val_loss: 0.3525
Epoch 4/30
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.7844 - loss: 0.4704 - val_accuracy: 0.8490 - val_loss: 0.3392
Epoch 5/30
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7999 - loss: 0.4501 - val_accuracy: 0.8565 - val_loss: 0.3329
Epoch 6/30
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8050 - loss: 0.4398 - val_accuracy: 0.8600 - val_loss: 0.3255
Epoch 7/30
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7996 - loss: 0.4466 - val_accuracy: 0.8660 - val_loss: 0.3207
